# Seldon Core Serving Example

Seldon Core comes installed with Kubeflow. The [Seldon Core documentation site](https://docs.seldon.io/projects/seldon-core/en/latest/) provides full documentation for running Seldon Core inference.

Seldon Core is an open source platform to deploy your machine learning models on Kubernetes at massive scale.It converts ML models (Tensorflow, Pytorch, H2o, etc.) or language wrappers (Python, Java, etc.) into production REST/GRPC microservices.

Seldon handles scaling to thousands of production machine learning models and provides advanced ML capabilities out of the box including Advanced Metrics, Request Logging, Explainers, Outlier Detectors, A/B Tests, Canaries and more.

### 1. Label current namespace so you can run inference tasks in it

In [ ]:
with open('/var/run/secrets/kubernetes.io/serviceaccount/namespace', 'r') as f:
    namespace = f.readline()

In [ ]:
!kubectl label namespace $namespace serving.kubeflow.org/inferenceservice=enabled

### 2. Create an Istio gateway in current namespace named kubeflow-gateway

In [ ]:
!kubectl create -n $namespace -f ./serving/seldon_gateway.yaml

**Note**: Execute all the below commands in your local environment

### 3. Create an example SeldonDeployment with a dummy model

In [ ]:
!kubectl create -n $namespace -f ./serving/seldon.yaml

### 4.Wait for state to become available

In [ ]:
!kubectl get seldondeployment seldon-model -n $namespace -o jsonpath='{.status.state}\n'

### 5. Port forward to the Istio gateway

In [ ]:
!kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8004:80

### 6. Send a prediction request

In [ ]:
!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8004/seldon/$namespace/seldon-model/api/v1.0/predictions -H "Content-Type: application/json"

You should be able to see a response like below:

```
{
  "meta": {
    "puid": "i2e1i8nq3lnttadd5i14gtu11j",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier_rest:1.3"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "ndarray": [[0.43782349911420193]]
  }
}
```

### 7. Canary Rollout

In [ ]:
!kubectl apply -n $namespace -f seldon_canary.yaml

Wait for state to become available

In [ ]:
!kubectl get seldondeployment seldon-model -n $namespace -o jsonpath='{.status.state}\n'

Run bash script to send 100 times prediction request

In [ ]:
%%bash

for i in {0..100};
do
  curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8004/seldon/$namespace/seldon-model/api/v1.0/predictions -H "Content-Type: application/json";
done

Check the traffic is split

In [ ]:
%%bash

default_count=$(kubectl logs $(kubectl get pods -n $namespace -l version=example -o jsonpath='{.items[0].metadata.name}') seldon-container-engine -n $namespace | grep "Calling grpc for transform-input" | wc -l)
canary_count=$(kubectl logs $(kubectl get pods -n $namespace -l version=canary-example -o jsonpath='{.items[0].metadata.name}') seldon-container-engine -n $namespace | grep "Calling grpc for transform-input" | wc -l)

echo "\nThe count of traffic routes to default: $default_count"
echo "The count of traffic routes to canary: $canary_count"

### 8. Clean up

In [ ]:
!kubectl delete -n $namespace seldondeployment seldon-model
!kubectl delete -n $namespace gateway kubeflow-gateway